In [ ]:
import dataclasses

import jax
import os 
import sys


root_dir = '/storage/qiguojunLab/caojinjin/codes/openpi/src'

# 将根目录添加到 sys.path
if root_dir not in sys.path:
    sys.path.append(root_dir)
    
from openpi.models import model as _model
from openpi.policies import droid_policy
from openpi.policies import policy_config as _policy_config
from openpi.shared import download
from openpi.training import config as _config
from openpi.training import data_loader as _data_loader

: 

# Policy inference

The following example shows how to create a policy from a checkpoint and run inference on a dummy example.

In [12]:
config = _config.get_config("pi0_fast_droid")
checkpoint_dir = download.maybe_download("s3://openpi-assets/checkpoints/pi0_fast_droid")

# Create a trained policy.
policy = _policy_config.create_trained_policy(config, checkpoint_dir)

# Run inference on a dummy example. This example corresponds to observations produced by the DROID runtime.
example = droid_policy.make_droid_example()
result = policy.infer(example)

# Delete the policy to free up memory.
del policy

print("Actions shape:", result["actions"].shape)

Some kwargs in processor config are unused and will not have any effect: time_horizon, action_dim, min_token, scale, vocab_size. 
Some kwargs in processor config are unused and will not have any effect: time_horizon, action_dim, min_token, scale, vocab_size. 


Actions shape: (10, 8)


# How to visualize the result trajectory?

# Working with a live model


The following example shows how to create a live model from a checkpoint and compute training loss. First, we are going to demonstrate how to do it with fake data.


In [6]:
config = _config.get_config("pi0_aloha_sim")

checkpoint_dir = download.maybe_download("s3://openpi-assets/checkpoints/pi0_aloha_sim")
key = jax.random.key(0)

# Create a model from the checkpoint.
model = config.model.load(_model.restore_params(checkpoint_dir / "params"))

# We can create fake observations and actions to test the model.
obs, act = config.model.fake_obs(), config.model.fake_act()

# Sample actions from the model.
loss = model.compute_loss(key, obs, act)
print("Loss shape:", loss.shape)

Loss shape: (1, 50)


Now, we are going to create a data loader and use a real batch of training data to compute the loss.

In [7]:
# Reduce the batch size to reduce memory usage.
config = dataclasses.replace(config, batch_size=2)

# Load a single batch of data. This is the same data that will be used during training.
# NOTE: In order to make this example self-contained, we are skipping the normalization step
# since it requires the normalization statistics to be generated using `compute_norm_stats`.
loader = _data_loader.create_data_loader(config, num_batches=1, skip_norm_stats=True)
obs, act = next(iter(loader))

# Sample actions from the model.
loss = model.compute_loss(key, obs, act)

# Delete the model to free up memory.
del model

print("Loss shape:", loss.shape)

Returning existing local_dir `/storage/qiguojunLab/qiguojun/.cache/huggingface/lerobot/lerobot/aloha_sim_transfer_cube_human` as remote repo cannot be accessed in `snapshot_download` (None).


FileNotFoundError: [Errno 2] No such file or directory: '/storage/qiguojunLab/qiguojun/.cache/huggingface/lerobot/lerobot/aloha_sim_transfer_cube_human/meta/info.json'